# Import Library

In [ ]:
import pandas as pd  # For data manipulation
import numpy as np   # For numerical operations
import pycaret
import matplotlib.pyplot as plt
import seaborn as sns
import shap
%matplotlib inline
from sklearn.model_selection import train_test_split  # For splitting data
from sklearn.model_selection import train_test_split  # For splitting data
from sklearn.preprocessing import StandardScaler, MinMaxScaler  # For scaling
from sklearn.impute import SimpleImputer  # For handling missing values
from sklearn.preprocessing import OneHotEncoder, LabelEncoder  # For encoding categorical data
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

import warning
warnings.filterwarnings("ignore")

# 1. Load Data

#### File Lokal: CSV, Excel, JSON, ZIP.
#### Database: SQL.
#### Big Data: Parquet, Feather, HDF5.
#### Online Resources: URL, API, Google Sheets.

### A. CSV

In [ ]:
df = pd.read_csv('your_dataset.csv')
df.head()  # Preview the first 5 rows of the dataset

### B. Excel

In [ ]:
# Load data from an Excel file
df = pd.read_excel('data.xlsx', sheet_name='Sheet1')


### C. JSON

In [ ]:
# Load data from a JSON file
df = pd.read_json('data.json')


### D. SQL

In [ ]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('database.db')

# Load data from an SQL query
df = pd.read_sql_query('SELECT * FROM tablename', conn)


### E. ZIP

In [ ]:
# Load data from a compressed ZIP file
df = pd.read_csv('data.zip', compression='zip')


### F. URL

In [ ]:
# Load data from a URL
df = pd.read_csv('https://url_to_data.com/data.csv')


### G. Google Sheets (API)

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Authenticate using Google API
scope = ['https://spreadsheets.google.com/feeds']
creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
client = gspread.authorize(creds)

# Open a sheet by name
sheet = client.open('SheetName').sheet1

# Get data as a list of lists
data = sheet.get_all_records()


### H. API (JSON)

In [ ]:
import requests

# Load data from a REST API
response = requests.get('https://api.example.com/data')
data = response.json()

df = pd.DataFrame(data)


### I. Parquet (HDFS/S3)

In [ ]:
# Load data from Parquet file
df = pd.read_parquet('data.parquet')


### J. Feather

In [ ]:
# Load data from a Feather file
df = pd.read_feather('data.feather')


### K. HDF5 (Hierarchical Data Format)

In [ ]:
# Load data from an HDF5 file
df = pd.read_hdf('data.h5', 'key_name')


### L. Pickle (Python Serializing)

In [ ]:
# Load data from a Pickle file
df = pd.read_pickle('data.pkl')


### M. arff (Attribute-Relation File Format)

In [ ]:
from scipy.io import arff
import pandas as pd

# Load ARFF file
data, meta = arff.loadarff('data.arff')

# Convert to DataFrame
df = pd.DataFrame(data)


# 2. Handling Missing Values

#### Penghapusan Baris/Kolom: Jika nilai hilang minor.
#### Pengisian Nilai: Menggunakan konstanta, statistik deskriptif, interpolasi, atau model.
#### Model Pembelajaran Mesin: Menggunakan KNN, regresi, atau algoritma khusus untuk mengisi nilai hilang.
#### Khusus Kategori atau Data Waktu: Mengisi berdasarkan kategori atau metode forward/backward fill.

### A. Detect Missing Values

In [ ]:
df.isnull().sum()  # Check how many missing values are in each column
persentase_hilang = df.isnull().sum() / df.shape[0] * 100



### B. Delete Row with Missing Values

In [ ]:
import pandas as pd

# Menghapus baris dengan nilai hilang
df = df.dropna()

# Menghapus kolom dengan nilai hilang
df = df.dropna(axis=1)


### C. Fill Missing Values (Constanta)

In [ ]:
# Mengisi nilai hilang dengan nol
df = df.fillna(0)

# Mengisi nilai hilang dengan nilai konstan lain
df = df.fillna(value={'column1': 0, 'column2': 'unknown'})


### D. Fill Missing Values (Mean, Median, Mode)

In [ ]:
# Mengisi nilai hilang dengan rata-rata kolom
df['column'] = df['column'].fillna(df['column'].mean())

# Mengisi nilai hilang dengan median kolom
df['column'] = df['column'].fillna(df['column'].median())

# Mengisi nilai hilang dengan modus kolom
df['column'] = df['column'].fillna(df['column'].mode()[0])


### F. Fill Missing Values (Category Value)

In [ ]:
# Mengisi nilai hilang dengan modus kategori
df['column'] = df.groupby('category')['column'].transform(lambda x: x.fillna(x.mode()[0]))


### G. Fill Missing Values (Pembobotan dan Penyesuaian)

In [ ]:
# Mengisi nilai hilang dalam data waktu dengan pembobotan
df['column'] = df['column'].fillna(method='ffill')  # Forward fill
df['column'] = df['column'].fillna(method='bfill')  # Backward fill


### H. Fill Missing Values (Interpolasi)

In [ ]:
# Mengisi nilai hilang dengan interpolasi linear
df = df.interpolate()

# Mengisi nilai hilang dengan interpolasi polinomial
df = df.interpolate(method='polynomial', order=2)


### I. Fill Missing Values (KNN)

In [ ]:
from sklearn.impute import KNNImputer

# Menggunakan KNN untuk imputasi
imputer = KNNImputer(n_neighbors=5)
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)


### J. Fill Missing Values (Regression Model)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Memisahkan data dengan nilai hilang
df_missing = df[df['column'].isna()]
df_complete = df.dropna(subset=['column'])

# Melatih model regresi
X = df_complete.drop(columns=['column'])
y = df_complete['column']
model = LinearRegression()
model.fit(X, y)

# Mengisi nilai hilang dengan prediksi model
X_missing = df_missing.drop(columns=['column'])
df_missing['column'] = model.predict(X_missing)
df = pd.concat([df_complete, df_missing])


### K. Fill Missing Values (IterativeImputer)

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Menggunakan IterativeImputer
imputer = IterativeImputer()
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)


### L. Fill Missing Values (Imputation Model Prediction)

In [ ]:
# Fill missing numerical data with the median
num_imputer = SimpleImputer(strategy='median')
df['num_column'] = num_imputer.fit_transform(df[['num_column']])

# Fill missing categorical data with the most frequent value
cat_imputer = SimpleImputer(strategy='most_frequent')
df['cat_column'] = cat_imputer.fit_transform(df[['cat_column']])

# 3. Encoding Categorical Data

#### Label Encoding: Digunakan untuk data kategorikal tanpa urutan.
#### One-Hot Encoding: Untuk model yang tidak mendukung data kategorikal.
#### Binary Encoding dan BaseN Encoding: Untuk mengurangi dimensi dibandingkan one-hot encoding.
#### Frequency dan Count Encoding: Menggunakan frekuensi kategori.
#### Target Encoding: Menggunakan rata-rata target per kategori.
#### Ordinal Encoding: Untuk data kategorikal yang memiliki urutan.
#### Hash Encoding dan Leave-One-Out Encoding: Untuk data besar atau data dengan banyak kategori.

### A. Label Encoding (For Ordinal Data)

In [ ]:
label_encoder = LabelEncoder()
df['category_encoded'] = label_encoder.fit_transform(df['category_column'])

------------------------------------------------------------------------------------------------------

for col in df_baru.columns:
    if df_baru[col].dtype == 'object':
        df_baru[col] = label_encoder.fit_transform(df_baru[col])
        df_baru[col] = df_baru[col].astype('int64')

### B. One-Hot Encoding (For Nominal Data)

In [ ]:
df = pd.get_dummies(df, columns=['category_column'], drop_first=True)

### C. Binary Encoding

In [ ]:
!pip install category_encoders
import category_encoders as ce

# Inisialisasi BinaryEncoder
encoder = ce.BinaryEncoder(cols=['categorical_column'])

# Menggunakan BinaryEncoder
df_encoded = encoder.fit_transform(df)


### D. Frequency Encoding

In [ ]:
# Menghitung frekuensi
frequency = df['categorical_column'].value_counts()

# Mengganti kategori dengan frekuensi
df['encoded_column'] = df['categorical_column'].map(frequency)


### E. Target Encoding (Mean Encoding)

In [ ]:
# Menghitung rata-rata target per kategori
mean_encoding = df.groupby('categorical_column')['target'].mean()

# Mengganti kategori dengan rata-rata target
df['encoded_column'] = df['categorical_column'].map(mean_encoding)


### F. Ordinal Encoding

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# Inisialisasi OrdinalEncoder
encoder = OrdinalEncoder()

# Menggunakan OrdinalEncoder pada kolom
df['encoded_column'] = encoder.fit_transform(df[['categorical_column']])


### G. BaseN Encoding

In [ ]:
import category_encoders as ce

# Inisialisasi BaseNEncoder dengan basis 3
encoder = ce.BaseNEncoder(base=3, cols=['categorical_column'])

# Menggunakan BaseNEncoder
df_encoded = encoder.fit_transform(df)


### H. Count Encoding

In [ ]:
# Menghitung jumlah kemunculan
count_encoding = df['categorical_column'].value_counts()

# Mengganti kategori dengan jumlah kemunculan
df['encoded_column'] = df['categorical_column'].map(count_encoding)


### I. Hash Encoding

In [ ]:
import category_encoders as ce

# Inisialisasi HashingEncoder dengan n_components
encoder = ce.HashingEncoder(n_components=8, cols=['categorical_column'])

# Menggunakan HashingEncoder
df_encoded = encoder.fit_transform(df)


### J. Leave-One-Out Encoding

In [ ]:
import category_encoders as ce

# Inisialisasi HashingEncoder dengan n_components
encoder = ce.HashingEncoder(n_components=8, cols=['categorical_column'])

# Menggunakan HashingEncoder
df_encoded = encoder.fit_transform(df)


# 4. Feature Scaling

#### Standardization: Menyediakan fitur dengan distribusi normal.
#### Min-Max Scaling: Mengubah rentang data ke [0, 1].
#### MaxAbs Scaling: Berguna untuk data dengan nilai positif dan negatif.
#### Robust Scaling: Mengurangi dampak outlier.
#### Quantile Transformation: Mengubah distribusi fitur.
#### Power Transformation: Membantu stabilisasi varians.
#### Normalizer: Menormalkan fitur ke norma unit.
#### Custom Scaling: Menyediakan kontrol penuh untuk scaling.
#### Binarization: Mengubah fitur ke format biner

In [ ]:
1. Standardization
Standardization (also called Z-score normalization) transforms the data to have a mean of 0 and a standard deviation of 1. This method is particularly useful when the data follows a Gaussian distribution (normal distribution).


2. Normalization
Normalization (also known as Min-Max scaling) rescales the feature to a fixed range, typically [0, 1]. This method is useful when you want to ensure that all features have the same scale, especially when using algorithms that are sensitive to the scale of input data (e.g., neural networks).

Key Differences
Purpose: Standardization centers the data around zero, while normalization rescales the data to a specific range.
Effect on Data: Standardization retains the distribution shape, while normalization compresses all values into a defined range.
Use Cases: Standardization is preferred for algorithms that assume Gaussian distributions, whereas normalization is useful for algorithms that rely on distance measures, like k-nearest neighbors or gradient descent optimization in neural networks.

Conclusion
Both standardization and normalization are essential preprocessing steps that can significantly affect the performance of machine learning models. The choice between them depends on the characteristics of your data and the requirements of the specific algorithm you are using.

### A. Standardization (Z-score Normalization)

In [ ]:
scaler = StandardScaler()
df[['num_column1', 'num_column2']] = scaler.fit_transform(df[['num_column1', 'num_column2']])

#Import: Kita mengimpor pandas untuk membuat dan mengelola DataFrame, dan StandardScaler dari sklearn.preprocessing untuk melakukan standardisasi.
#DataFrame: Kita membuat DataFrame dengan dua kolom numerik (num_column1 dan num_column2) serta satu kolom kategori.
#StandardScaler: Kita membuat objek StandardScaler untuk menstandardisasi data.
#fit_transform: Fungsi ini digunakan untuk menghitung mean dan standard deviation dari kolom yang diberikan, dan kemudian menerapkan transformasi standardisasi pada data.

#Kegunaan StandardScaler
#Menghilangkan Skala: StandardScaler mengubah data sehingga memiliki mean 0 dan standar deviasi 1. Ini sangat berguna ketika fitur memiliki skala yang berbeda dan dapat mempengaruhi hasil model.

#Meningkatkan Kinerja Model: Banyak algoritma, seperti K-Nearest Neighbors (KNN) dan Support Vector Machines (SVM), sangat sensitif terhadap skala data. Standardisasi dapat meningkatkan kinerja model dengan memberikan kontribusi yang lebih seimbang dari setiap fitur.

#Meningkatkan Konvergensi: Dalam algoritma optimasi, seperti Gradient Descent yang digunakan dalam pembelajaran mendalam, standardisasi dapat mempercepat konvergensi karena semua fitur berada pada skala yang sama.

#Memudahkan Interpretasi: Dengan memiliki fitur yang terstandardisasi, hasil analisis atau model dapat lebih mudah diinterpretasikan karena tidak terpengaruh oleh skala yang berbeda antara fitur.

### B. Min-Max Scaling

In [ ]:
min_max_scaler = MinMaxScaler()
df[['num_column1', 'num_column2']] = min_max_scaler.fit_transform(df[['num_column1', 'num_column2']])

#Import: Kita mengimpor pandas untuk membuat dan mengelola DataFrame, dan MinMaxScaler dari sklearn.preprocessing untuk melakukan normalisasi.
#DataFrame: Kita membuat DataFrame dengan dua kolom numerik (num_column1 dan num_column2) serta satu kolom kategori.
#MinMaxScaler: Kita membuat objek MinMaxScaler untuk melakukan normalisasi pada data.
#fit_transform: Fungsi ini menghitung nilai minimum dan maksimum dari kolom yang diberikan, dan kemudian menerapkan transformasi normalisasi sehingga semua nilai berada dalam rentang [0, 1].


#Kegunaan MinMaxScaler
#Skala Data Konsisten: Normalisasi dengan MinMaxScaler memastikan bahwa semua fitur berada dalam rentang yang sama, biasanya [0, 1]. Ini membantu algoritma yang bergantung pada jarak, seperti K-Nearest Neighbors (KNN) dan Support Vector Machines (SVM).

#Menghindari Dominasi Fitur: Ketika fitur memiliki skala yang berbeda, fitur dengan skala yang lebih besar dapat mendominasi hasil model. Normalisasi membantu menghindari masalah ini dengan memberikan kontribusi yang seimbang dari setiap fitur.

#Meningkatkan Kinerja Model: Dengan fitur yang dinormalisasi, algoritma pembelajaran mesin dapat beroperasi lebih efektif, meningkatkan akurasi dan kecepatan konvergensi.

#Mempermudah Interpretasi: Hasil model menjadi lebih mudah diinterpretasikan ketika semua fitur berada dalam rentang yang sama, sehingga analisis lebih intuitif.

#Bermanfaat untuk Data yang Tidak Terdistribusi Normal: Jika data tidak mengikuti distribusi normal, normalisasi dengan MinMaxScaler dapat menjadi pilihan yang lebih baik dibandingkan standardisasi.

### D. MaxAbs Scaling

In [ ]:
from sklearn.preprocessing import MaxAbsScaler

# Inisialisasi MaxAbsScaler
scaler = MaxAbsScaler()

# Fit dan transformasi data
scaled_data = scaler.fit_transform(df[['feature_column']])


#Import: Kita mengimpor pandas untuk membuat dan mengelola DataFrame, serta MaxAbsScaler dari sklearn.preprocessing untuk melakukan normalisasi.
#DataFrame: Kita membuat DataFrame dengan satu kolom numerik (feature_column) yang memiliki nilai positif dan negatif.
#MaxAbsScaler: Kita membuat objek MaxAbsScaler untuk melakukan normalisasi berdasarkan nilai maksimum absolut.
#fit_transform: Fungsi ini menghitung nilai maksimum absolut dari kolom yang diberikan, dan kemudian menerapkan transformasi sehingga semua nilai berada dalam rentang [-1, 1].


#Kegunaan MaxAbsScaler
#Mempertahankan Tanda: MaxAbsScaler menjaga tanda asli dari nilai (positif/negatif). Ini sangat berguna ketika data memiliki arah yang signifikan dan Anda ingin mempertahankan informasi tersebut.

#Skala Data yang Konsisten: Dengan menggunakan MaxAbsScaler, semua fitur dinormalisasi dalam rentang [-1, 1], sehingga memudahkan komparasi antar fitur.

#Cocok untuk Data Spars: MaxAbsScaler sangat bermanfaat untuk dataset yang memiliki banyak nilai nol (sparse data), seperti yang sering ditemui dalam pengolahan teks dan data kategori. Normalisasi ini tidak mengubah nilai nol dan tetap mempertahankan bentuk data asli.

#Meningkatkan Kinerja Model: Dengan fitur yang dinormalisasi, algoritma pembelajaran mesin dapat beroperasi lebih efektif, terutama algoritma yang sensitif terhadap skala data, seperti SVM atau neural networks.

#Meningkatkan Interpretasi: Normalisasi membuat interpretasi hasil analisis lebih mudah, karena semua fitur sekarang berada dalam skala yang sama.

### E. Robust Scaling

In [ ]:
from sklearn.preprocessing import RobustScaler

# Inisialisasi RobustScaler
scaler = RobustScaler()

# Fit dan transformasi data
scaled_data = scaler.fit_transform(df[['feature_column']])


#Import: Kita mengimpor pandas untuk membuat dan mengelola DataFrame, serta RobustScaler dari sklearn.preprocessing untuk melakukan normalisasi.
#DataFrame: Kita membuat DataFrame dengan satu kolom numerik (feature_column) yang berisi beberapa nilai yang tinggi (outlier) dan nilai normal.
#RobustScaler: Kita membuat objek RobustScaler untuk melakukan normalisasi yang tahan terhadap outlier.
#fit_transform: Fungsi ini menghitung median dan kuartil dari kolom yang diberikan, lalu menerapkan transformasi normalisasi berdasarkan median dan rentang interkuartil (IQR).


#Kegunaan RobustScaler
#Tahan Terhadap Outlier: RobustScaler menggunakan median dan interquartile range (IQR) untuk normalisasi, yang membuatnya kurang sensitif terhadap outlier dibandingkan metode lain seperti Min-Max Scaling atau Standardization.

#Mempertahankan Distribusi Data: Dengan menggunakan median dan IQR, RobustScaler mempertahankan distribusi data asli, sehingga memberikan hasil yang lebih representatif ketika data mengandung outlier.

#Skala Data Konsisten: Normalisasi dengan RobustScaler menghasilkan skala yang konsisten, dengan nilai yang biasanya berada dalam rentang [-1, 1], yang memudahkan interpretasi dan komparasi.

#Cocok untuk Data dengan Distribusi Asimetris: RobustScaler sangat berguna ketika data tidak terdistribusi normal dan memiliki banyak outlier, sehingga dapat digunakan dalam berbagai situasi.

#Meningkatkan Kinerja Model: Dengan fitur yang dinormalisasi dengan cara yang tahan terhadap outlier, algoritma pembelajaran mesin dapat beroperasi lebih efektif, meningkatkan akurasi model.

### F. Quantile Transformation

In [ ]:
from sklearn.preprocessing import QuantileTransformer

# Inisialisasi QuantileTransformer
scaler = QuantileTransformer(output_distribution='uniform')

# Fit dan transformasi data
scaled_data = scaler.fit_transform(df[['feature_column']])


#Import: Kita mengimpor pandas untuk membuat dan mengelola DataFrame, serta QuantileTransformer dari sklearn.preprocessing untuk melakukan normalisasi.
#DataFrame: Kita membuat DataFrame dengan satu kolom numerik (feature_column) yang berisi beberapa nilai, termasuk outlier.
#QuantileTransformer: Kita membuat objek QuantileTransformer dengan parameter output_distribution='uniform' untuk mendistribusikan nilai ke dalam rentang [0, 1].
#fit_transform: Fungsi ini menghitung kuantil dari kolom yang diberikan, lalu menerapkan transformasi sehingga semua nilai didistribusikan secara merata dalam rentang [0, 1].


#Kegunaan QuantileTransformer
#Transformasi ke Distribusi Uniform: QuantileTransformer mendistribusikan nilai-nilai ke dalam distribusi uniform, yang membuatnya cocok untuk algoritma yang memerlukan data dalam skala tertentu.

#Tahan Terhadap Outlier: Karena menggunakan kuantil, QuantileTransformer kurang sensitif terhadap outlier dibandingkan metode normalisasi tradisional. Ini membuatnya ideal untuk data yang memiliki nilai ekstrem.

#Cocok untuk Data Tidak Terdistribusi Normal: Metode ini dapat digunakan pada data yang tidak mengikuti distribusi normal, membantu untuk mempersiapkan data dengan cara yang lebih sesuai untuk analisis atau pemodelan lebih lanjut.

#Meningkatkan Kinerja Model: Dengan mendistribusikan nilai secara merata, QuantileTransformer dapat meningkatkan kinerja model machine learning, terutama pada algoritma yang sensitif terhadap skala data.

#Mempermudah Interpretasi: Normalisasi dengan QuantileTransformer membuat interpretasi hasil analisis lebih mudah, karena semua fitur berada dalam skala yang sama dan lebih terdistribusi secara merata.

### G. Power Transformation (Box-Cox & Yeo-Johnson)

In [ ]:
from sklearn.preprocessing import PowerTransformer

# Inisialisasi PowerTransformer dengan metode Yeo-Johnson
scaler = PowerTransformer(method='yeo-johnson')

# Fit dan transformasi data
scaled_data = scaler.fit_transform(df[['feature_column']])


#Import: Kita mengimpor pandas untuk membuat dan mengelola DataFrame, serta PowerTransformer dari sklearn.preprocessing untuk melakukan transformasi.
#DataFrame: Kita membuat DataFrame dengan satu kolom numerik (feature_column) yang berisi beberapa nilai, termasuk outlier.
#PowerTransformer: Kita membuat objek PowerTransformer dengan parameter method='yeo-johnson', yang memungkinkan kita untuk menerapkan transformasi yang dapat menangani nilai nol dan negatif.
#fit_transform: Fungsi ini menghitung parameter yang diperlukan (seperti lambda untuk transformasi) dan menerapkan transformasi pada data.


#Kegunaan PowerTransformer
#Mengatasi Asimetri Data: PowerTransformer, khususnya dengan metode Yeo-Johnson, dirancang untuk mengubah data yang tidak terdistribusi normal menjadi lebih mendekati distribusi normal. Ini sangat berguna untuk algoritma yang mengasumsikan distribusi normal.

#Menangani Outlier: Metode ini juga dapat membantu mengurangi dampak dari outlier pada analisis dan pemodelan, menjadikan data lebih stabil.

#Kesesuaian dengan Data Negatif: Berbeda dengan Box-Cox, Yeo-Johnson dapat digunakan pada data yang mengandung nilai negatif dan nol, sehingga lebih fleksibel dalam berbagai jenis dataset.

#Meningkatkan Kinerja Model: Dengan mendekatkan distribusi data ke distribusi normal, PowerTransformer dapat meningkatkan kinerja model machine learning yang bergantung pada asumsi normalitas.

#Mempermudah Interpretasi: Transformasi data sehingga lebih mendekati distribusi normal dapat membuat hasil analisis lebih mudah dipahami dan diinterpretasikan.

### H. Normalizer

In [ ]:
from sklearn.preprocessing import Normalizer

# Inisialisasi Normalizer dengan norma L2
scaler = Normalizer(norm='l2')

# Fit dan transformasi data
scaled_data = scaler.fit_transform(df[['feature_column']])


#Import: Kita mengimpor pandas untuk membuat dan mengelola DataFrame, serta Normalizer dari sklearn.preprocessing untuk melakukan normalisasi.
#DataFrame: Kita membuat DataFrame dengan satu kolom numerik (feature_column) yang berisi beberapa nilai.
#Normalizer: Kita membuat objek Normalizer dengan parameter norm='l2', yang akan menormalkan data sehingga panjang vektornya (norma) menjadi 1.
#fit_transform: Fungsi ini menghitung norma dari kolom yang diberikan dan menerapkan normalisasi berdasarkan norma L2, menghasilkan vektor yang memiliki panjang 1.


#Kegunaan Normalizer
#Normalisasi Vektor: Normalizer digunakan untuk menormalkan setiap sampel (baris) dalam dataset sehingga panjang vektornya menjadi 1. Ini penting dalam konteks di mana sudut antar vektor lebih penting daripada magnitudonya.

#Cocok untuk Algoritma Berbasis Jarak: Normalisasi sangat berguna untuk algoritma seperti K-Nearest Neighbors (KNN) atau clustering, di mana jarak antara data merupakan faktor kunci dalam klasifikasi atau pengelompokan.

#Meningkatkan Kinerja Model: Dengan menggunakan normalisasi, kita dapat mengurangi pengaruh fitur dengan skala besar yang dapat mendominasi hasil analisis, sehingga meningkatkan kinerja model.

#Menjaga Informasi Arah: Normalization dengan norma L2 tidak mengubah arah data tetapi hanya menskalakan magnitudo, sehingga informasi tentang hubungan antara data tetap terjaga.

#Menghindari Overfitting: Normalisasi juga dapat membantu mengurangi risiko overfitting dengan memastikan bahwa semua fitur memiliki pengaruh yang seimbang pada model.

### I. Unit Vector Scaling

In [ ]:
from sklearn.preprocessing import Normalizer

# Inisialisasi Normalizer dengan norma L2
scaler = Normalizer(norm='l2')

# Fit dan transformasi data
scaled_data = scaler.fit_transform(df[['feature_column']])


#Import: Mengimpor pandas untuk manipulasi DataFrame dan Normalizer dari sklearn.preprocessing untuk normalisasi.
#DataFrame: Membuat DataFrame dengan kolom feature_column yang berisi beberapa nilai numerik.
#Normalizer: Membuat objek Normalizer dengan parameter norm='l2', yang akan menormalkan setiap baris dalam DataFrame sehingga norma (panjang) vektornya menjadi 1.
#fit_transform: Menghitung norma dari kolom yang diberikan dan menerapkan normalisasi, menghasilkan array yang memiliki panjang 1 untuk setiap baris.


#Kegunaan Normalizer
#Normalisasi Vektor: Normalizer mengubah setiap baris menjadi vektor dengan panjang 1, penting dalam konteks di mana sudut antar vektor lebih penting daripada magnitudonya.

#Cocok untuk Algoritma Berbasis Jarak: Sangat berguna untuk algoritma seperti K-Nearest Neighbors (KNN) dan clustering, di mana jarak antar data sangat berpengaruh.

#Meningkatkan Kinerja Model: Mengurangi pengaruh fitur dengan skala besar yang dapat mendominasi hasil analisis, meningkatkan kinerja model secara keseluruhan.

#Mempertahankan Informasi Arah: Menggunakan norma L2 tidak mengubah arah data tetapi hanya menskalakan magnitudo, sehingga hubungan antara data tetap terjaga.

#Mencegah Overfitting: Dengan normalisasi, risiko overfitting dapat berkurang karena semua fitur akan memiliki pengaruh yang lebih seimbang.

### J. Custom Scaling

In [ ]:
import numpy as np

# Fungsi custom untuk scaling
def custom_scaling(x):
    return (x - np.min(x)) / (np.max(x) - np.min(x))

# Menerapkan custom scaling
df['scaled_column'] = custom_scaling(df['feature_column'])

#Import: Mengimpor pandas untuk manipulasi DataFrame dan numpy untuk operasi numerik.
#DataFrame: Membuat DataFrame dengan kolom feature_column yang berisi beberapa nilai numerik.
#Fungsi Custom Scaling:
#Fungsi custom_scaling(x) menerima array x dan mengembalikan nilai yang telah diskalakan menggunakan rumus min-max.
#Rumus: (𝑥−min(𝑥))/(max(𝑥)−min(𝑥))(x−min(x))/(max(x)−min(x)) mengubah setiap nilai dalam x menjadi rentang [0, 1].
#Menerapkan Scaling: Fungsi custom_scaling diterapkan pada kolom feature_column, dan hasilnya disimpan dalam kolom baru scaled_column.

#Kegunaan Custom Scaling
#Kustomisasi Proses Scaling: Dengan menggunakan fungsi kustom, kita dapat menyesuaikan rumus atau metode scaling sesuai dengan kebutuhan spesifik.

#Menangani Data dengan Cara Unik: Kita dapat melakukan transformasi yang tidak terbatas pada metode standar seperti Min-Max atau Z-score, misalnya, menyesuaikan skala berdasarkan kriteria tertentu.

#Pendidikan dan Pemahaman: Membuat fungsi kustom membantu dalam memahami mekanisme scaling dan bagaimana pengaruhnya terhadap data, yang bermanfaat untuk pembelajaran dan eksplorasi.

#Fleksibilitas: Dapat dengan mudah dimodifikasi untuk menerapkan teknik lain atau menggabungkan beberapa metode scaling dalam satu fungsi.

### K. Binarization

In [ ]:
from sklearn.preprocessing import Binarizer

# Inisialisasi Binarizer dengan threshold
scaler = Binarizer(threshold=0.5)

# Fit dan transformasi data
scaled_data = scaler.fit_transform(df[['feature_column']])


#Import: Mengimpor pandas untuk manipulasi DataFrame dan Binarizer dari sklearn.preprocessing untuk binarisasi.
#DataFrame: Membuat DataFrame dengan kolom feature_column yang berisi beberapa nilai numerik, termasuk nilai di bawah dan di atas threshold.
#Binarizer: Membuat objek Binarizer dengan parameter threshold=0.5, yang akan mengubah semua nilai di atas 0.5 menjadi 1 dan yang di bawahnya menjadi 0.
#fit_transform: Menerapkan binarisasi pada kolom yang diberikan, menghasilkan array yang berisi 0 atau 1.


#Kegunaan Binarizer
#Membuat Data Kategori: Binarizer sangat berguna untuk mengubah data kontinu menjadi kategori. Ini penting dalam konteks model klasifikasi, di mana fitur perlu dalam bentuk biner.

#Meningkatkan Interpretabilitas: Mengubah nilai menjadi 0 dan 1 dapat memudahkan pemahaman hasil analisis dan membuat visualisasi lebih sederhana.

#Memudahkan Penggunaan dalam Algoritma Klasifikasi: Banyak algoritma machine learning, seperti Logistic Regression, memerlukan fitur dalam bentuk biner untuk pemodelan.

#Penanganan Data Besar: Binarisasi dapat membantu mereduksi kompleksitas data, terutama ketika kita hanya peduli pada kehadiran atau ketidakhadiran suatu fitur.

#Fleksibilitas Threshold: Pengguna dapat menentukan threshold yang sesuai dengan konteks data, memungkinkan penyesuaian yang lebih baik terhadap analisis yang diinginkan.

# 5. Splitting Data into Training and Testing Sets

#### K-Fold Cross-Validation: Menggunakan seluruh data untuk pelatihan dan pengujian, memberikan penilaian model yang lebih robust.
#### Stratified K-Fold: Menjaga distribusi target dalam setiap fold.
#### Time Series Split: Menjaga urutan waktu dalam data.
#### LOOCV: Membagi data dengan satu data sebagai uji pada setiap iterasi.
#### Group K-Fold: Mempertahankan kelompok tertentu dalam data.
#### Shuffle Split: Membagi data secara acak dengan proporsi yang ditentukan.
#### Custom Split: Pembagian data berdasarkan kriteria khusus.
#### Bootstrap Sampling: Menggunakan sampling dengan pengembalian untuk data.

### A. Split Standar

In [ ]:
X = df.drop('target_column', axis=1)
y = df['target_column']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### B. K-Fold Cross-Validation

In [ ]:
from sklearn.model_selection import KFold

# Inisialisasi KFold dengan jumlah fold
kf = KFold(n_splits=5)

# Menggunakan KFold untuk membagi data
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]


### C. Stratified K-Fold Cross-Validation

In [ ]:
from sklearn.model_selection import StratifiedKFold

# Inisialisasi StratifiedKFold
skf = StratifiedKFold(n_splits=5)

# Menggunakan StratifiedKFold untuk membagi data
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]


### D. Time Series Split

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

# Inisialisasi TimeSeriesSplit dengan jumlah fold
tscv = TimeSeriesSplit(n_splits=5)

# Menggunakan TimeSeriesSplit untuk membagi data
for train_index, test_index in tscv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]


### E. Leave-One-Out Cross-Validation (LOOCV)

In [ ]:
from sklearn.model_selection import LeaveOneOut

# Inisialisasi LeaveOneOut
loo = LeaveOneOut()

# Menggunakan LeaveOneOut untuk membagi data
for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]


### F. Group K-Fold

In [ ]:
from sklearn.model_selection import GroupKFold

# Inisialisasi GroupKFold dengan jumlah fold
gkf = GroupKFold(n_splits=5)

# Menggunakan GroupKFold untuk membagi data
for train_index, test_index in gkf.split(X, y, groups=groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]


### G. Shuffle Split

In [ ]:
from sklearn.model_selection import ShuffleSplit

# Inisialisasi ShuffleSplit
ss = ShuffleSplit(n_splits=5, test_size=0.2)

# Menggunakan ShuffleSplit untuk membagi data
for train_index, test_index in ss.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]


### H. Custom Split

In [ ]:
# Contoh custom split berdasarkan kriteria waktu
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


### I. Bootstrap Sampling

In [ ]:
from sklearn.utils import resample

# Menggunakan resample untuk membuat subset
X_train, X_test, y_train, y_test = resample(X, y, n_samples=0.8, random_state=42)


# 6. Handling Outliers

#### Identifikasi dan Penghapusan: Menggunakan metode statistik atau visual untuk mengidentifikasi dan menghapus outliers.
#### Transformasi Data: Menggunakan transformasi matematis untuk mengurangi efek outliers.
#### Robust Methods: Metode scaling dan winsorizing yang mengurangi dampak outliers.
#### Imputasi Outliers: Mengganti atau interpolasi nilai outliers untuk menjaga konsistensi data.
#### Algorithm-Based Methods: Menggunakan algoritma machine learning untuk mendeteksi dan menangani outliers.

### A. Using Z-score to Remove Outliers

In [ ]:
from scipy import stats

z_scores = np.abs(stats.zscore(df[['num_column1', 'num_column2']]))
df_cleaned = df[(z_scores < 3).all(axis=1)]

### B. IQR (Interquartile Range)

In [ ]:
Q1 = df['feature_column'].quantile(0.25)
Q3 = df['feature_column'].quantile(0.75)
IQR = Q3 - Q1
# Menentukan batas bawah dan atas
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
# Mengidentifikasi outliers
outliers = (df['feature_column'] < lower_bound) | (df['feature_column'] > upper_bound)
# Menghapus outliers
df_no_outliers = df[~outliers]


### C. Visualisasi (Boxplot)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Membuat boxplot
sns.boxplot(x=df['feature_column'])
plt.show()


### D. Log Transformation

In [ ]:
import numpy as np

# Mengaplikasikan log transformation
df['log_feature'] = np.log1p(df['feature_column'])


### E. Square Root Transformation

In [ ]:
# Mengaplikasikan square root transformation
df['sqrt_feature'] = np.sqrt(df['feature_column'])


### F. Robust Scaling

In [ ]:
from sklearn.preprocessing import RobustScaler

# Inisialisasi RobustScaler
scaler = RobustScaler()

# Fit dan transformasi data
df[['scaled_feature']] = scaler.fit_transform(df[['feature_column']])


### G. Trimming/ Winsorizing

In [ ]:
from scipy.stats import mstats

# Winsorizing data
df['winsorized_feature'] = mstats.winsorize(df['feature_column'], limits=[0.05, 0.05])


### H. Imputasi dengan Nilai Median atau Mean

In [ ]:
# Mengganti outliers dengan median
median_value = df['feature_column'].median()
df['feature_column'] = np.where(outliers, median_value, df['feature_column'])


### I. Imputasi dengan Interpolation

In [ ]:
# Interpolasi data
df['interpolated_feature'] = df['feature_column'].interpolate()


### J. Isolation Forest

In [ ]:
from sklearn.ensemble import IsolationForest

# Inisialisasi IsolationForest
iso_forest = IsolationForest(contamination=0.01)

# Fit model
outliers = iso_forest.fit_predict(df[['feature_column']])
# Mengidentifikasi outliers
df_no_outliers = df[outliers == 1]


### K. One-Class SVM

In [ ]:
from sklearn.svm import OneClassSVM

# Inisialisasi OneClassSVM
ocsvm = OneClassSVM(gamma='auto', nu=0.01)

# Fit model
outliers = ocsvm.fit_predict(df[['feature_column']])
# Mengidentifikasi outliers
df_no_outliers = df[outliers == 1]


# 7. Feature Selection

#### Model-Based Feature Importance: Menggunakan output model untuk menilai pentingnya fitur.
#### Recursive Feature Elimination (RFE): Metode iteratif untuk menghapus fitur yang tidak penting.
#### Mutual Information: Mengukur ketergantungan antara fitur dan target.
#### Correlation Matrix: Mengidentifikasi fitur yang berkorelasi dengan target atau antar fitur.
#### L1 Regularization (Lasso): Menggunakan regularisasi untuk memilih fitur penting.
#### Genetic Algorithms: Metode berbasis evolusi untuk memilih fitur optimal.
#### Feature Selection using AutoML: Mengotomatisasi proses feature selection dan tuning.
#### Unsupervised Feature Selection: Menggunakan teknik unsupervised untuk memilih fitur penting.

### A. Removing Low Variance Features

In [ ]:
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(threshold=0.1)
df_high_variance = selector.fit_transform(df)

### B. Correlation Matrix

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

corr_matrix = df.corr()
plt.figure(figsize=(12,8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.show()

### C. Model-Based Feature Importance

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Inisialisasi dan fit model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Mendapatkan feature importances
importances = model.feature_importances_

# Mengurutkan fitur berdasarkan kepentingan
indices = np.argsort(importances)[::-1]
for i in indices:
    print(f"Feature {i}: {importances[i]}")


### D. Permutation Feature Importance

In [ ]:
from sklearn.inspection import permutation_importance

# Fit model
model.fit(X_train, y_train)

# Menghitung permutation feature importance
results = permutation_importance(model, X_test, y_test, scoring='accuracy')
importances = results.importances_mean


### E. Recursive Feature Elimination (RFE)

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# Inisialisasi model dan RFE
model = LogisticRegression()
rfe = RFE(model, n_features_to_select=5)

# Fit RFE
fit = rfe.fit(X_train, y_train)

# Mendapatkan fitur yang terpilih
selected_features = fit.support_


### F. Mutual Information

In [ ]:
from sklearn.feature_selection import mutual_info_classif

# Menghitung mutual information
mi = mutual_info_classif(X_train, y_train)

# Mengurutkan fitur berdasarkan mutual information
indices = np.argsort(mi)[::-1]
for i in indices:
    print(f"Feature {i}: {mi[i]}")


### G. L1 Regularization (Lasso)

In [ ]:
from sklearn.linear_model import Lasso

# Inisialisasi Lasso
lasso = Lasso(alpha=0.01)

# Fit model
lasso.fit(X_train, y_train)

# Mendapatkan koefisien fitur
coefficients = lasso.coef_


### H. Genetic Algorithms

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFECV

# Inisialisasi RFECV dengan Logistic Regression
selector = RFECV(LogisticRegression(), step=1, cv=5)
selector = selector.fit(X_train, y_train)

# Mendapatkan fitur yang terpilih
selected_features = selector.support_


### I. Feature Selection using AutoML

In [ ]:
from tpot import TPOTClassifier

# Inisialisasi TPOT
tpot = TPOTClassifier(verbosity=2)

# Fit model
tpot.fit(X_train, y_train)

# Mendapatkan pipeline terbaik
print(tpot.fitted_pipeline_)


### J. Unsupervised Feature Selection

In [ ]:
from sklearn.decomposition import PCA

# Inisialisasi PCA
pca = PCA(n_components=5)

# Fit PCA
X_pca = pca.fit_transform(X_train)

# Mendapatkan komponen utama
components = pca.components_


# 8. Binning Continuous Features

#### Equal Width Binning: Membagi rentang data menjadi bin dengan lebar yang sama.
#### Equal Frequency Binning: Membagi data sehingga setiap bin memiliki jumlah data yang sama.
#### Custom Binning: Menetapkan interval bin secara manual.
#### K-Means Binning: Menggunakan clustering untuk menentukan bin.
#### Decision Tree Binning: Menentukan bin berdasarkan model decision tree.
#### Equal Width Binning with Variable Bin Width: Menggunakan bin dengan lebar bervariasi berdasarkan distribusi data.
#### Binning with Aggregation: Menghitung statistik agregasi per bin.

### A. Equal Width Binning

In [ ]:
import pandas as pd
import numpy as np

# Contoh data
df = pd.DataFrame({'feature': np.random.randn(100)})

# Mendefinisikan jumlah bin
num_bins = 5

# Menggunakan cut untuk binning
df['binned_feature'] = pd.cut(df['feature'], bins=num_bins, labels=False)

print(df.head())


### B. Equal Frequency Binning

In [ ]:
import pandas as pd
import numpy as np

# Contoh data
df = pd.DataFrame({'feature': np.random.randn(100)})

# Mendefinisikan jumlah bin
num_bins = 5

# Menggunakan qcut untuk binning
df['binned_feature'] = pd.qcut(df['feature'], q=num_bins, labels=False)

print(df.head())


### C. Custom Binning

In [ ]:
import pandas as pd
import numpy as np

# Contoh data
df = pd.DataFrame({'feature': np.random.randn(100)})

# Mendefinisikan interval bin
bins = [-np.inf, -1, 0, 1, np.inf]
labels = ['Very Low', 'Low', 'High', 'Very High']

# Menggunakan cut untuk binning
df['binned_feature'] = pd.cut(df['feature'], bins=bins, labels=labels)

print(df.head())


### D. K-Means Binning

In [ ]:
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np

# Contoh data
df = pd.DataFrame({'feature': np.random.randn(100)})

# Menentukan jumlah cluster (bin)
num_bins = 5

# Inisialisasi dan fit KMeans
kmeans = KMeans(n_clusters=num_bins, random_state=0).fit(df[['feature']])

# Menambahkan label cluster ke dataframe
df['binned_feature'] = kmeans.labels_

print(df.head())


### E. Decision Tree Binning

In [ ]:
from sklearn.tree import DecisionTreeRegressor
import pandas as pd
import numpy as np

# Contoh data
df = pd.DataFrame({'feature': np.random.randn(100), 'target': np.random.randn(100)})

# Inisialisasi dan fit DecisionTreeRegressor
tree = DecisionTreeRegressor(max_leaf_nodes=5, random_state=0)
tree.fit(df[['feature']], df['target'])

# Mendapatkan batas bin
bins = np.sort(tree.tree_.threshold[tree.tree_.threshold != -2])
bins = np.append(bins, np.inf)  # Menambahkan batas atas

# Menggunakan cut untuk binning
df['binned_feature'] = pd.cut(df['feature'], bins=bins, include_lowest=True)

print(df.head())


### F. Equal Width Binning with Variable Bin Width

In [ ]:
import pandas as pd
import numpy as np

# Contoh data
df = pd.DataFrame({'feature': np.random.randn(100)})

# Mendefinisikan lebar bin berdasarkan persentil
percentiles = [0, 0.1, 0.3, 0.5, 0.7, 0.9, 1]
bins = np.percentile(df['feature'], q=percentiles)

# Menggunakan cut untuk binning
df['binned_feature'] = pd.cut(df['feature'], bins=bins, labels=False)

print(df.head())


### G. Binning with Aggregation

In [ ]:
import pandas as pd
import numpy as np

# Contoh data
df = pd.DataFrame({'feature': np.random.randn(100)})

# Menggunakan qcut untuk binning
df['binned_feature'] = pd.qcut(df['feature'], q=5, labels=False)

# Menghitung statistik agregasi per bin
bin_stats = df.groupby('binned_feature').agg({'feature': ['mean', 'std', 'count']})

print(bin_stats)


# 9. Data Balancing (Handling Imbalanced Classes)

### Oversampling Methods: SMOTE, ADASYN, Random Oversampling.

#### Kegunaan: Menambah data pada kelas minoritas.
#### Catatan: Memerlukan penanganan overfitting dan mungkin memperbesar ukuran dataset.
### Undersampling Methods: Random Undersampling, Tomek Links, Edited Nearest Neighbors.

#### Kegunaan: Mengurangi data pada kelas mayoritas.
#### Catatan: Dapat menyebabkan kehilangan informasi dan memerlukan pemilihan parameter yang hati-hati.
### Hybrid Methods: SMOTE-ENN, SMOTE-Tomek Links.

#### Kegunaan: Menggabungkan teknik oversampling dan undersampling.
#### Catatan: Menyediakan pendekatan seimbang untuk menangani data ketidakseimbangan.

## OVERSAMPLING

### A. Using Random Oversampling

In [ ]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

### B. Using SMOTE (Synthetic Minority Over-sampling Technique)

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

### C. ADASYN (Adaptive Synthetic Sampling Approach for Imbalanced Learning)

In [ ]:
from imblearn.over_sampling import ADASYN

# Inisialisasi ADASYN
adasyn = ADASYN(sampling_strategy='auto', random_state=0)

# Fit dan transformasi data
X_resampled, y_resampled = adasyn.fit_resample(X_train, y_train)


## UNDERSAMPLING

### A. Random Undersampling

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

# Inisialisasi RandomUnderSampler
rus = RandomUnderSampler(sampling_strategy='auto', random_state=0)

# Fit dan transformasi data
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)


### B. Tomek Links

In [ ]:
from imblearn.under_sampling import TomekLinks

# Inisialisasi TomekLinks
tomek = TomekLinks(sampling_strategy='auto')

# Fit dan transformasi data
X_resampled, y_resampled = tomek.fit_resample(X_train, y_train)


### C. Edited Nearest Neighbors (ENN)

In [ ]:
from imblearn.under_sampling import EditedNearestNeighbours

# Inisialisasi EditedNearestNeighbours
enn = EditedNearestNeighbours(sampling_strategy='auto')

# Fit dan transformasi data
X_resampled, y_resampled = enn.fit_resample(X_train, y_train)


## HYBRID

### A. SMOTE-ENN

In [ ]:
from imblearn.combine import SMOTEENN

# Inisialisasi SMOTEENN
smote_enn = SMOTEENN(sampling_strategy='auto', random_state=0)

# Fit dan transformasi data
X_resampled, y_resampled = smote_enn.fit_resample(X_train, y_train)


### B. SMOTE-Tomek Links

In [ ]:
from imblearn.combine import SMOTETomek

# Inisialisasi SMOTETomek
smote_tomek = SMOTETomek(sampling_strategy='auto', random_state=0)

# Fit dan transformasi data
X_resampled, y_resampled = smote_tomek.fit_resample(X_train, y_train)


# 10. saving and Loading Preprocessed Data

#### Untuk data kecil hingga menengah: CSV, Excel, dan JSON.
#### Untuk data besar dan efisien: Parquet, Feather, dan HDF5.
#### Untuk penyimpanan objek Python: Pickle dan Joblib.
#### Untuk mendukung kueri SQL: SQLite.

### A. CSV (Comma-Separated Values)

In [ ]:
df.to_csv('processed_data.csv', index=False)

### B. Excel

In [ ]:
# Menyimpan dataframe ke file Excel
df.to_excel('preprocessed_data.xlsx', index=False)


### C. JSON (JavaScript Object Notation)

In [ ]:
# Menyimpan dataframe ke JSON
df.to_json('preprocessed_data.json', orient='records')


### D. SQLite (Database)

In [ ]:
import sqlite3

# Koneksi ke database SQLite
conn = sqlite3.connect('preprocessed_data.db')

# Menyimpan dataframe ke database
df.to_sql('table_name', conn, if_exists='replace', index=False)


### E. Pickle

In [ ]:
# Menyimpan dataframe ke dalam file pickle
df.to_pickle('preprocessed_data.pkl')


### F. HDF5 (Hierarchical Data Format)

In [ ]:
# Menyimpan dataframe ke HDF5
df.to_hdf('preprocessed_data.h5', key='df', mode='w')


### G. Feather

In [ ]:
# Menyimpan dataframe ke feather
df.to_feather('preprocessed_data.feather')


### H. Parquet

In [ ]:
# Menyimpan dataframe ke Parquet
df.to_parquet('preprocessed_data.parquet')


### I. Joblib

In [ ]:
import joblib

# Menyimpan dataframe ke file joblib
joblib.dump(df, 'preprocessed_data.joblib')
